In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import pathlib, json
import torch
import os
import pandas as pd
import numpy as np
import pandas as pd
import pickle, sys
import warnings
from tqdm import tqdm
from sklearn.model_selection import KFold

sys.path.append('../')
sys.path.append('../../')

from recipes.dataset import MCPASDataset

# from analysis_util import display_mat_from_ind
# from analysis_util import convert_len

from Bio.PDB import *

# from pdb_util import get_chain_list, calc_dist, remove_HOH, get_structure_from_id, from_str_to_chain_names, get_residues_from_names
# from pdb_util import get_cdrs_from_anarci
# from pdb_util import get_seqs_from_residues
warnings.filterwarnings(action='once')



# PDB Command

In [3]:
sys.path.append('../streamlit/')
from streamlit_utils import *

loading....


2022-03-31 09:47:19.682 
  command:

    streamlit run /Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
../streamlit/streamlit_utils.py:28: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/20220211_191954_DICT_PDBID_2_Atten12.pickle'>
  def pickleload(p): return pickle.load(open(p, "rb"))
../streamlit/streamlit_utils.py:28: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/20220211_191954_DICT_PDBID_2_MELTDIST.pickle'>
  def pickleload(p): return pickle.load(open(p, "rb"))
../streamlit/streamlit_utils.py:28: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/kyoheikoyama/workspace/tcrpred/analysis/DICT_PDB_Result/20220211_191954_DICT_PDBID_2_model_out.pickle'>
  def pickleload(p): return pickle.load(open(p, "rb"))
../streamlit/streamlit_utils.py:28: Resour

ending... loading....


100%|████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.39it/s]
21it [00:00, 50.14it/s]

2OL3 has only delta-chain and beta-chain


61it [00:01, 36.85it/s]

4QRR has only delta-chain and beta-chain
5JHD has only delta-chain and beta-chain
5JZI has only delta-chain and beta-chain


79it [00:01, 42.18it/s]
/Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [4]:
from Bio.PDB.PDBParser import PDBParser

parser = PDBParser()
structure = parser.get_structure(pdbid, f"./../analysis/analysis1_alldata/pdb//pdb{pdbid.lower()}.ent")


/Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 13784.
  warnings.warn(
/Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 13789.
  warnings.warn(
/Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 13794.
  warnings.warn(
/Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 13804.
  warnings.warn(


In [5]:
pdbids = list(DICT_PDBID_2_CDRS.keys())

In [6]:
KKK = 1
for p in pdbids:
    if len(DICT_PDBID_2_CDRS[p][KKK])==0: continue
    i = DICT_PDBID_2_CHAINNAMES[p]
    print(DICT_PDBID_2_CDRS[p][KKK][0].get_full_id()[2], i[KKK])

E E
J J, L
E E, J
B B
E E
E E, J, O, T
E E, J
E E, J, O, T
E E
E E
E E, J, O, T
E E
E E, J, O, T
E E
E E
E E
H H
E E
E E
F F, H
E E
J J
F F, M
D D
E E
E E
E E, J
E E, J
B B, F
E E
F F, H
F F, M
F F, M
E E
B B
E E
E E
E E
E E
D D
D D, H
D D, E
B B
E E
E E
F F, H
E E
F F, H
D D, H
F F, H
E E, J, P, U
E E
B B
E E
J J, L, P, F
B B, F
E E
E E
D D
E E, J
E E
E E, J
E E, J
E E


In [7]:
def add_commnad(Pymol_COMMAND, text):
    return Pymol_COMMAND + text + '\n'

def get_command(pdbid):
    Pymol_COMMAND = ''
    chain_list = list([c.get_id() for c in structure.get_chains()])
    
    residues_chain_cdr_alpha = DICT_PDBID_2_CDRS[pdbid][0]
    residues_chain_cdr_beta = DICT_PDBID_2_CDRS[pdbid][1]
    chain2_cdr3_pdb_pos = [r.get_full_id()[3][1] for r in residues_chain_cdr_beta]
    chain1_cdr3_pdb_pos = [r.get_full_id()[3][1] for r in residues_chain_cdr_alpha]

    ALPHA_CHAIN_NAME, BETA_CHAIN_NAME, EPITOPE_CHAIN_NAME = \
        DICT_PDBID_2_CDRS[pdbid][0][0].get_full_id()[2], DICT_PDBID_2_CDRS[pdbid][1][0].get_full_id()[2], DICT_PDBID_2_CDRS[pdbid][2][0].get_full_id()[2]
    MHC_CHAIN_NAME = [c for c in chain_list if c not in ALPHA_CHAIN_NAME+BETA_CHAIN_NAME+EPITOPE_CHAIN_NAME]

    attn_output_weights1_list, attn_output_weights2_list = DICT_PDBID_2_Atten12[pdbid]
    attention_residue_index = []
    attention_residue_headnum = []
    for headi in range(4):
        atten = attn_output_weights2_list[headi]
        good_vals = (atten > (atten.values.ravel().mean() + 5.0 * atten.values.ravel().std())).any()
        attention_residue_index += [i for i,v in enumerate(good_vals.values) if v]
        attention_residue_headnum += [headi for _ in range(sum(good_vals.values))]

    residues_all = residues_chain_cdr_alpha + [None] + residues_chain_cdr_beta
    attention_residue_alpha = [residues_all[i] for i in attention_residue_index if i<len(residues_chain_cdr_alpha)]
    attention_residue_alpha_head = [attention_residue_headnum[i] for i, ri in enumerate(attention_residue_index) if ri<len(residues_chain_cdr_alpha)]
    attention_residue_beta = [residues_all[i] for i in attention_residue_index if i>=len(residues_chain_cdr_beta)]
    attention_residue_beta_head = [attention_residue_headnum[i] for i, ri in enumerate(attention_residue_index) if ri>=len(residues_chain_cdr_alpha)]
    attention_pos_alpha = [r.get_full_id()[3][1] for r in attention_residue_alpha]
    attention_pos_beta = [r.get_full_id()[3][1] for r in attention_residue_beta]

    ###. Base commands ################################################################
    
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'fetch {pdbid};')
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'set seq_view, 1;')
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'bg_color white;')
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'set_color blue60, [ 0, 0, 110 ];')
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'set_color blue30, [ 0, 0, 20 ];')
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'set_color red60, [ 110, 0, 0 ];')
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'set_color red30, [ 20, 0, 0 ];')
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'hide all;')
    ##############################################################################
    
    
    ### Base chains and colors ###################################################
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'select beta, chain {BETA_CHAIN_NAME} and not solvent;')
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'select alpha, chain {ALPHA_CHAIN_NAME} and not solvent;')
    if len(MHC_CHAIN_NAME)==1:
        mhc_chain = f'chain {MHC_CHAIN_NAME[0]}'
    else:
        mhc_chain = ''
        for m in MHC_CHAIN_NAME:
            mhc_chain += f'chain {m} or '
        mhc_chain = mhc_chain[:-4]
    Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'select mhc, ({mhc_chain}) and not solvent;')    
    coloring_commands = ['show ribbon, alpha;', 'color salmon, alpha;', 'show ribbon, beta;', 'color cyan, beta;', 
                'show ribbon, mhc;', 'color grey70, mhc;']
    for com in coloring_commands:
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, com)
    ##############################################################################
    
    
    if len(BETA_CHAIN_NAME)==1:
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel beta_cdr3, (chain {BETA_CHAIN_NAME} and resi {chain2_cdr3_pdb_pos[0]}:{chain2_cdr3_pdb_pos[-1]});')
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'set cartoon_side_chain_helper, on')
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'show sticks, beta_cdr3;')
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'util.cbag beta_cdr3;')
    else:
        for b in BETA_CHAIN_NAME.split(', '):
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel beta_cdr3, (chain {b} and resi {chain2_cdr3_pdb_pos[0]}:{chain2_cdr3_pdb_pos[-1]});')
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'set cartoon_side_chain_helper, on')
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'show sticks, beta_cdr3;')
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'util.cbag beta_cdr3;')
        
    
    if len(ALPHA_CHAIN_NAME)==1:        
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel alpha_cdr3, (chain {ALPHA_CHAIN_NAME} and resi {chain1_cdr3_pdb_pos[0]}:{chain1_cdr3_pdb_pos[-1]});')
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'set cartoon_side_chain_helper, on')
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'show sticks, alpha_cdr3;')
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'util.cbag alpha_cdr3;')
    else:
        for a in ALPHA_CHAIN_NAME.split(', '):
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel alpha_cdr3, (chain {a} and resi {chain1_cdr3_pdb_pos[0]}:{chain1_cdr3_pdb_pos[-1]});')
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'set cartoon_side_chain_helper, on')
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'show sticks, alpha_cdr3;')
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'util.cbag alpha_cdr3;')

    if len(EPITOPE_CHAIN_NAME)==1:
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel epitope, chain {EPITOPE_CHAIN_NAME} and not solvent;')
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'show sticks, epitope;')
    else:
        for e in EPITOPE_CHAIN_NAME.split(', '):
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel epitope, chain {e} and not solvent;')
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, 'show sticks, epitope;')
    
    
    
    bond_distance_commands = ["util.cbay epitope;",
            "select cdr3, alpha_cdr3 or beta_cdr3;",
            "select tcr, alpha or beta;",
            "dist H_cdr_p, cdr3, epitope, mode=2;",
            "hide labels, H_cdr_p;",
            "color black, H_cdr_p;",
            "dist H_cdr_tcr, cdr3, tcr, mode=2;",
            "hide labels, H_cdr_tcr;",
            "color grey, H_cdr_tcr;",]
    
    for com in bond_distance_commands:
        Pymol_COMMAND = add_commnad(Pymol_COMMAND, com)

    '''Attention'''
    if len(ALPHA_CHAIN_NAME)==1:
        for hi, ap in zip(attention_residue_alpha_head, attention_pos_alpha):
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel atten_a_head{hi}, (chain {ALPHA_CHAIN_NAME} and resi {ap});')
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'color pink, atten_a_head{hi};')
    else:
        for a in ALPHA_CHAIN_NAME.split(', '):
            for hi, ap in zip(attention_residue_alpha_head, attention_pos_alpha):
                Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel atten_a_head{hi}, (chain {a} and resi {ap});')
                Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'color pink, atten_a_head{hi};')
        
    if len(BETA_CHAIN_NAME)==1:
        for hi, ap in zip(attention_residue_beta_head, attention_pos_beta):
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel atten_b_head{hi}, (chain {BETA_CHAIN_NAME} and resi {ap});')
            Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'color pink, atten_b_head{hi};')
    else:
        for b in BETA_CHAIN_NAME.split(', '):
            for hi, ap in zip(attention_residue_beta_head, attention_pos_beta):
                Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'sel atten_b_head{hi}, (chain {b} and resi {ap});')
                Pymol_COMMAND = add_commnad(Pymol_COMMAND, f'color pink, atten_b_head{hi};')            
    return Pymol_COMMAND

In [33]:
print(get_command(pdbid='1LP9'))

fetch 1LP9;
set seq_view, 1;
bg_color white;
set_color blue60, [ 0, 0, 110 ];
set_color blue30, [ 0, 0, 20 ];
set_color red60, [ 110, 0, 0 ];
set_color red30, [ 20, 0, 0 ];
hide all;
select beta, chain F and not solvent;
select alpha, chain E and not solvent;
select mhc, (chain A or chain B or chain D) and not solvent;
show ribbon, alpha;
color salmon, alpha;
show ribbon, beta;
color cyan, beta;
show ribbon, mhc;
color grey70, mhc;
sel beta_cdr3, (chain F and resi 92:108);
set cartoon_side_chain_helper, on
show sticks, beta_cdr3;
util.cbag beta_cdr3;
sel alpha_cdr3, (chain E and resi 90:106);
set cartoon_side_chain_helper, on
show sticks, alpha_cdr3;
util.cbag alpha_cdr3;
sel epitope, chain C and not solvent;
show sticks, epitope;
util.cbay epitope;
select cdr3, alpha_cdr3 or beta_cdr3;
select tcr, alpha or beta;
dist H_cdr_p, cdr3, epitope, mode=2;
hide labels, H_cdr_p;
color black, H_cdr_p;
dist H_cdr_tcr, cdr3, tcr, mode=2;
hide labels, H_cdr_tcr;
color grey, H_cdr_tcr;
sel atten_a_

# Save command into file

In [34]:
positive_pdbid_list = [k for k,v in DICT_PDBID_2_model_out.items() if v[-1]>0.5]

In [35]:
!mkdir ./../streamlit/pymolcommand_ribons

mkdir: ./../streamlit/pymolcommand_ribons: File exists


In [36]:
for pdbid in positive_pdbid_list:
    if pdbid not in DICT_PDBID_2_Atten12:
        continue
    with open(f'./../streamlit/pymolcommand_ribons/{pdbid}.txt', 'w') as f:
        f.writelines(get_command(pdbid))


In [40]:
def res_to_alphabetseq(rlist):
    return ''.join([AACODES_DICT.get(r.get_resname(), 'X') for r in rlist])

DICT_PDBID_2_SEQ = {k:[res_to_alphabetseq(v) for v in vv] for k,vv in DICT_PDBID_2_CDRS.items()}

In [47]:
df_seq_paired = pd.DataFrame(DICT_PDBID_2_SEQ).T
df_seq_paired = df_seq_paired.loc[positive_pdbid_list]
df_seq_paired.drop_duplicates(subset=[0,1,2]).shape

(39, 3)

In [37]:
len([pdbid for pdbid in positive_pdbid_list if pdbid not in DICT_PDBID_2_Atten12]),\
len([pdbid for pdbid in positive_pdbid_list if pdbid in DICT_PDBID_2_Atten12]),\
len(positive_pdbid_list)

(0, 47, 47)